In [64]:
from konlpy.tag import Twitter
from ipywidgets import FloatProgress
from IPython.display import display
from torch.autograd import Variable
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm
from torch.nn import functional as F
from torch import nn
import pickle
import torch
import os

twitter = Twitter()
if not os.path.exists("./data"):
    os.mkdir("./data")

In [58]:
with open("data/data_deep.pkl", "rb") as f:
    index2voca = pickle.load(f)
    voca2index = pickle.load(f)
    train_X = pickle.load(f)
    train_y = pickle.load(f)
    test_X = pickle.load(f)
    test_y = pickle.load(f)

vocaNum = len(index2voca)
trainNum = len(train_y)
testNum = len(test_y)

embedding_dim = 200
hidden_size = 200
num_layer = 2
bidirectional = True
epoch = 10
batch_size = 32
print_iter = 2000

mode = 'CPU'

In [59]:
train_X = torch.from_numpy(train_X).long()
train_y = torch.from_numpy(train_y).view(-1, 1)
test_X = torch.from_numpy(test_X).long()
test_y = torch.from_numpy(test_y).view(-1, 1)

train = TensorDataset(train_X, train_y)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=1)
test = TensorDataset(test_X, test_y)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)

In [79]:
class RNNReg(nn.Module):
    def __init__(self, num_layer=2, hidden_size=200, bidirectional=True):
        super(RNNReg, self).__init__()
        self.num_layer = num_layer
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        self.linear_dim1 = (
            int(self.bidirectional) + 1) * num_layer * self.hidden_size
        self.linear_dim2 = 10
        self.embedding = nn.Embedding(vocaNum, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=self.hidden_size,
            num_layers=self.num_layer,
            batch_first=True,
            bidirectional=self.bidirectional)
        self.fc1 = nn.Linear(
            in_features=self.linear_dim1, out_features=self.linear_dim2)
        self.fc2 = nn.Linear(in_features=self.linear_dim2, out_features=1)
        self.dropout = nn.Dropout()

    def forward(self, x):
        x = self.embedding(x)
        _, hidden = self.gru(x)
        output = hidden.permute(1, 0, 2).contiguous()
        output = output.view(-1, self.linear_dim1)
        output = F.relu(self.fc1(output))
        output = self.dropout(output)
        output = self.fc2(output)
        return output

In [80]:
reg = RNNReg(num_layer, hidden_size, bidirectional)
if mode == 'GPU':
    reg.cuda()
print(reg)

criterion = nn.MSELoss()
opt = torch.optim.Adadelta(reg.parameters())

RNNReg (
  (embedding): Embedding(20000, 200, padding_idx=0)
  (gru): GRU(200, 200, num_layers=2, batch_first=True, bidirectional=True)
  (fc1): Linear (800 -> 10)
  (fc2): Linear (10 -> 1)
  (dropout): Dropout (p = 0.5)
)


In [ ]:
num_iter = int(trainNum / batch_size)
print(f"num_iter: {num_iter}")
f = FloatProgress(min=0, max=print_iter)
display(f)

for e in range(epoch):
    for i, (batch_X, batch_y) in enumerate(train_loader):
        f.value += 1
        
        if mode == 'GPU':
            batch_X = Variable(batch_X).cuda()
            batch_y = Variable(batch_y).cuda()
        else:
            batch_X = Variable(batch_X)
            batch_y = Variable(batch_y)
        
        predict = reg(batch_X)
        
        opt.zero_grad()
        loss = criterion(predict, batch_y)
        loss.backward()
        clip_grad_norm(reg.parameters(), 0.5)
        opt.step()

        if i % print_iter == 0 or i == num_iter - 1:
            f.value = 0
            print("batch: {}, iteration: {}, loss: {}\n".format(e, i, loss.data.mean()))
            
torch.save(reg.state_dict(), 'data/rnn_regression.pkl')

In [82]:
criterion = nn.L1Loss()
average_loss = 0
num_iter = int(testNum / batch_size)
f = FloatProgress(min=0, max=num_iter)
display(f)
for i, (batch_X, batch_y) in enumerate(test_loader):
    f.value += 1
    
    if mode == 'GPU':
        batch_X = Variable(batch_X).cuda()
        batch_y = Variable(batch_y).cuda()
    else:
        batch_X = Variable(batch_X)
        batch_y = Variable(batch_y)

    predict = reg(batch_X)
    loss = criterion(predict, batch_y)
    average_loss += loss.data.mean()

average_loss /= num_iter
print("test error(MAE): {}".format(average_loss))

test error(MAE): 0.1876827024555958


In [ ]:
reg = RNNReg()

if mode == 'CPU':
    reg.load_state_dict(torch.load("data/rnn_regression.pkl", map_location=lambda storage, loc: storage))
else:
    reg.load_state_dict(torch.load("data/rnn_regression.pkl"))

In [75]:
reg.eval()
test_seq = u"팝콘 소리 때문에 집중이 안됐다 ㅡㅡ"
test_vec = torch.zeros(1, train_X.shape[1]).long()
for i, w in enumerate(twitter.morphs(test_seq)):
    test_vec[0, i] = voca2index.get(w, voca2index['<UNK>'])

test_vec = Variable(test_vec)
predict = reg(test_vec)
print("predicted: {}".format(predict.data.tolist()[0][0]))

predicted: 0.271027147769928
